In [16]:
from model import Chatbot, Encoder, Decoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from pathlib import Path
from unidecode import unidecode
import numpy as np

In [17]:
data_dir = Path('data')

with open(data_dir/'preguntas.txt', 'r', encoding='utf-8') as f:
    questions = f.readlines()

with open(data_dir/'respuestas.txt', 'r', encoding='utf-8') as f:
    answers = f.readlines()

print(f"El archivo 'preguntas.txt' contiene {len(questions)} preguntas")

El archivo 'preguntas.txt' contiene 171 preguntas


In [18]:
START_TOKEN = '<START> '
END_TOKEN = ' <END>'

processed_questions = list(map(lambda x: unidecode(x.lower()), questions))
processed_answers = list(map(lambda x: START_TOKEN + unidecode(x.lower().strip()) + END_TOKEN, answers))

print(processed_answers[0])

<START> "rosebud" <END>


In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_questions + processed_answers)
vocab_size = len(tokenizer.word_index)
print(f"There are {vocab_size} unique words")

There are 1064 unique words


In [20]:
# Preparing Encoder Input
tokenized_questions = tokenizer.texts_to_sequences(processed_questions)
max_input_length = max(list(map(len, tokenizer.texts_to_sequences(processed_questions))))
encoder_input = np.array(pad_sequences(tokenized_questions, maxlen=max_input_length, padding='post'))
print(f"Encoder input data shape: {encoder_input.shape}")

# Preparing Decoder Input
tokenized_answers = tokenizer.texts_to_sequences(processed_answers)
max_output_length = max(list(map(len, tokenizer.texts_to_sequences(processed_answers))))
decoder_input = np.array(pad_sequences(tokenized_answers, maxlen=max_input_length, padding='post'))
print(f"Decoder input data shape: {decoder_input.shape}")

# Preparing Target Output
tokenized_output = list(map(lambda x: x[1:], tokenized_answers))
padded_target = pad_sequences(tokenized_output, maxlen=max_input_length, padding='post')
decoder_target = np.array(to_categorical(padded_target))
print(f"Decoder target data shape: {decoder_target.shape}")

Encoder input data shape: (171, 31)
Decoder input data shape: (171, 31)
Decoder target data shape: (171, 31, 1065)


In [21]:
model = Chatbot(vocab_size + 1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input, decoder_input], decoder_target, batch_size=10, epochs=150)

Epoch 1/150
18/18 [==============================] - 11s 154ms/step - loss: 1.0552 - accuracy: 0.2725
Epoch 2/150
18/18 [==============================] - 3s 154ms/step - loss: 0.7114 - accuracy: 0.3380
Epoch 3/150
18/18 [==============================] - 3s 168ms/step - loss: 0.6443 - accuracy: 0.3979
Epoch 4/150
18/18 [==============================] - 3s 154ms/step - loss: 0.6220 - accuracy: 0.3935
Epoch 5/150
18/18 [==============================] - 3s 154ms/step - loss: 0.5442 - accuracy: 0.4294
Epoch 6/150
18/18 [==============================] - 3s 161ms/step - loss: 0.5653 - accuracy: 0.4057
Epoch 7/150
18/18 [==============================] - 3s 160ms/step - loss: 0.5246 - accuracy: 0.4293
Epoch 8/150
18/18 [==============================] - 3s 164ms/step - loss: 0.5211 - accuracy: 0.4183
Epoch 9/150
18/18 [==============================] - 3s 160ms/step - loss: 0.5031 - accuracy: 0.4198
Epoch 10/150
18/18 [==============================] - 3s 160ms/step - loss: 0.4680 - accur

In [23]:
model.save_weights(Path('model')/'model.hdf5')

In [34]:
def preprocess_input(text: str):
    text = text.lower()
    text = unidecode(text)
    tokenized_text = tokenizer.texts_to_sequences([text])
    padded_text = pad_sequences(tokenized_text, maxlen=max_input_length, padding='post')

    return padded_text

In [25]:
#question = input("Hola, hazme una pregunta")
#model = Chatbot(vocab_size + 1)
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.train_on_batch([encoder_input[:1], decoder_input[:1]], decoder_target[:1])
#model.load_weights(Path('model')/'model.hdf5')

In [36]:
question = 'Qué pelicula de Alfred Hitchcock se rodo como si fuera una toma continua'
empty_target_seq = np.array([[tokenizer.word_index['start']]])
stop_condition = False
answer = ''

proc_question = preprocess_input(question)

h, c = model.encoder.predict(proc_question)

while not stop_condition:
    decoder_output, h, c = model.decoder.predict([empty_target_seq, h, c])
    sampled_word_index = np.argmax( decoder_output[0, -1, :] )
    sampled_word = tokenizer.index_word.get(sampled_word_index, None)
    if sampled_word == 'end' or len(answer.split()) > max_output_length:
        stop_condition = True
    else:
        answer += f"{sampled_word} "
    

    empty_target_seq = np.array([[sampled_word_index]])
    enc_stats = [h, c]


print(answer)


la soga 
